#### The purpose of this notebook is to determine how structures are numbered in the cluster.X.all.txt files produced by sampcon. I determined that A structures are indexed by their frame number in A.rmf3. B structures are indexed by their frame number in B.rmf3 + len(A.rmf3). I used manual calculations of cluster precision to verify. 

In [ ]:
import pandas as pd
from pathlib import Path
import IMP 
import IMP.atom
import IMP.rmf
import RMF
import random
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
hdbscan_clust_file = Path("~/mtorc2/exp_8/66/analysis/analysis.367036/traj/A_cluster1_10000.csv")
hdbscan_clust_df = pd.read_csv(hdbscan_clust_file)
hdbscan_clust_df.head()


In [ ]:
samp_clust_file = Path("~/mtorc2/exp_8/66/analysis/analysis.367036/sampcon/cluster.2.all.txt")
samp_clust_df = pd.read_csv(samp_clust_file, header=None)
samp_clust_df.head()


In [ ]:
samp_clust_A_df = samp_clust_df[samp_clust_df[0]<10000]
samp_clust_A_df.head()

In [ ]:
samp_clust_B_df = samp_clust_df[samp_clust_df[0]>=10000].copy()
samp_clust_B_df[0] = samp_clust_B_df[0].apply(lambda x: x - 10000)
    
samp_clust_B_df.head()



In [ ]:
rmf_file_clust = Path(Path.home(), "mtorc2/exp_8/66/analysis/analysis.367036/sampcon/cluster.2/cluster_center_model.rmf3")
fh_clust = RMF.open_rmf_file_read_only(str(rmf_file_clust))
m_clust = IMP.Model()
h_clust = IMP.rmf.create_hierarchies(fh_clust, m_clust)[0]
    

In [ ]:
xyzs_1 = IMP.core.XYZs(IMP.core.get_leaves(h_clust))
xyzs_2 = IMP.core.XYZs(IMP.core.get_leaves(h_clust))
IMP.atom.get_rmsd(xyzs_1, xyzs_2)

In [ ]:
len(IMP.atom.Selection(h_clust, resolution=0).get_selected_particle_indexes())



In [ ]:
len(IMP.atom.Selection(h_clust, resolution=10).get_selected_particle_indexes())



In [ ]:
IMP.atom.Selection(h_clust).get_selected_particle_indexes()




In [ ]:
sel_1 = IMP.atom.Selection(h_clust, resolution=10).get_selected_particles()
sel_2 = IMP.atom.Selection(h_, resolution=10, particle_type).get_selected_particles()

xyzs_1 = IMP.core.XYZs(sel_1)
xyzs_2 = IMP.core.XYZs(sel_2)



In [ ]:
sel_res_1 = IMP.atom.Selection(h_clust, molecule="MSIN1", resolution=1)
sel_res_10 = IMP.atom.Selection(h_clust, molecule="MSIN1", resolution=10)
# sel_res_1.set_difference(sel_res_10)
# print(len(sel_res_1.get_selected_particle_indexes()))



In [ ]:
pids = [pid for pid in sel_res_1.get_selected_particle_indexes() if "bead" not in m_clust.get_particle_name(pid)]
print(len(pids))



In [ ]:
for pid in sel_res_1.get_selected_particle_indexes(): 
    print(m_clust.get_particle_name(pid))

In [ ]:
pids_res_1 = [pid for pid in sel_res_10.get_selected_particle_indexes() if IMP.core.XYZ.get_is_setup(m_clust, pid)]



In [ ]:
print(len(pids_res_1))

In [ ]:
print(m_clust.get_particle_name(sel_res_10.get_selected_particle_indexes()[0]))


In [ ]:
test_sel_1 = IMP.atom.Selection(h_clust, molecule="MTOR")
test_sel_2 = IMP.atom.Selection(h_clust, molecule="RICTOR")
test_sel_1.set_difference(test_sel_2)
print(len(test_sel_1.get_selected_particle_indexes()))

In [ ]:
for pid in sel_res_10.get_selected_particle_indexes():
    print(m_clust.get_particle_name(pid))
    

In [ ]:
sel_res_10.set_difference(sel_res_1)

In [ ]:
print(len(IMP.core.get_leaves(h_clust)))

In [ ]:
clust_precision = 0 

rmf_file = Path(Path.home(), "mtorc2/exp_8/66/analysis/analysis.367036/hdbscan/A_1_10000.rmf3")
fh = RMF.open_rmf_file_read_only(str(rmf_file))
m = IMP.Model()
h = IMP.rmf.create_hierarchies(fh, m)[0]

for i in range(1, len(samp_clust_A_df)):
    if i % 100 == 0:
        print(i)
        
    rmf_id = int(samp_clust_A_df.iloc[i, 0])
    IMP.rmf.load_frame(fh, RMF.FrameID(rmf_id))

    sel_1 = IMP.atom.Selection(h_clust, resolution=1)
    sel_2 = IMP.atom.Selection(h, resolution=1)

    pids_1 = [pid for pid in sel_1.get_selected_particle_indexes()]
    pids_2 = [pid for pid in sel_2.get_selected_particle_indexes()]
    
    xyzs_1 = [IMP.core.XYZ(m_clust, pid) for pid in pids_1]
    xyzs_2 = [IMP.core.XYZ(m, pid) for pid in pids_2]
    
    t = IMP.core.get_transformation_aligning_first_to_second(
        xyzs_2, xyzs_1)
    
    rmsd = IMP.atom.get_rmsd_transforming_first(t, xyzs_2, xyzs_1)
    
    clust_precision = clust_precision + rmsd
    
del fh 
    
rmf_file = Path(Path.home(), "mtorc2/exp_8/66/analysis/analysis.367036/hdbscan/B_1_10000.rmf3")
fh = RMF.open_rmf_file_read_only(str(rmf_file))
m = IMP.Model()
h = IMP.rmf.create_hierarchies(fh, m)[0]

for i in range(1, len(samp_clust_B_df)):
    if i % 100 == 0:
        print(i)
        
    rmf_id = int(samp_clust_B_df.iloc[i, 0])
    IMP.rmf.load_frame(fh, RMF.FrameID(rmf_id))

    sel_1 = IMP.atom.Selection(h_clust, resolution=1)
    sel_2 = IMP.atom.Selection(h, resolution=1)

    pids_1 = [pid for pid in sel_1.get_selected_particle_indexes()]
    pids_2 = [pid for pid in sel_2.get_selected_particle_indexes()]

    xyzs_1 = [IMP.core.XYZ(m_clust, pid) for pid in pids_1]
    xyzs_2 = [IMP.core.XYZ(m, pid) for pid in pids_2]

    t = IMP.core.get_transformation_aligning_first_to_second(
        xyzs_2, xyzs_1)
    
    rmsd = IMP.atom.get_rmsd_transforming_first(t, xyzs_2, xyzs_1)
    
    clust_precision = clust_precision + rmsd
    
del fh


In [ ]:
clust_precision

## Iterate through all cluster structures and check XL satisfaction.

In [ ]:
xl_files = list() 
xl_files.append(Path(Path.home(), "mtorc2/data/xlms/57.csv"))
xl_files.append(Path(Path.home(), "mtorc2/data/xlms/A.csv"))
xl_files.append(Path(Path.home(), "mtorc2/data/xlms/akt.csv"))

frames = list() 
for file in xl_files: 
    frames.append(pd.read_csv(file))

xl_df = pd.concat(frames)
xl_df["sat"] = False
xl_df = xl_df.drop(columns=["Unnamed: 0"])

print(len(xl_df))
xl_df.head()


In [ ]:
test_sel = IMP.atom.Selection(h, molecule="KINASE", residue_index=38)
pid = test_sel.get_selected_particle_indexes()[0]
xyz = IMP.core.XYZ(m, pid).get_coordinates()
xyz

In [ ]:
clust_precision = 0 

rmf_file = Path(Path.home(), "mtorc2/exp_8/66/analysis/analysis.367036/hdbscan/A_1_10000.rmf3")
fh = RMF.open_rmf_file_read_only(str(rmf_file))
m = IMP.Model()
h = IMP.rmf.create_hierarchies(fh, m)[0]

for i in range(len(samp_clust_A_df)):
    if i % 100 == 0:
        print(i)
        
    rmf_id = int(samp_clust_A_df.iloc[i, 0])
    IMP.rmf.load_frame(fh, RMF.FrameID(rmf_id))

    for j in range(len(xl_df)): 
        prot1, res1, prot2, res2 = xl_df.iloc[j,0:4]

        sel1 = IMP.atom.Selection(h, molecule=prot1, residue_index=res1)
        pid1 = sel1.get_selected_particle_indexes()[0]
        xyz1 = IMP.core.XYZ(m, pid1).get_coordinates()

        sel2 = IMP.atom.Selection(h, molecule=prot2, residue_index=res2)
        pid2 = sel2.get_selected_particle_indexes()[0]
        xyz2 = IMP.core.XYZ(m, pid).get_coordinates()    

        dist = np.linalg.norm(np.array(xyz1)-np.array(xyz2))
        if dist < 35: 
            if not xl_df.iloc[j, xl_df.columns.get_loc("sat")]: 
                print(prot1, res1, prot2, res2, i)
    
            xl_df.iloc[j, xl_df.columns.get_loc("sat")] = True
#         else: 
#             xl_df.iloc[j, xl_df.columns.get_loc("sat")] = True
    
del fh 
    
# rmf_file = Path(Path.home(), "mtorc2/exp_8/66/analysis/analysis.367036/hdbscan/B_1_10000.rmf3")
# fh = RMF.open_rmf_file_read_only(str(rmf_file))
# m = IMP.Model()
# h = IMP.rmf.create_hierarchies(fh, m)[0]

# for i in range(len(samp_clust_B_df)):
#     if i % 100 == 0:
#         print(i)
        
#     for i in range(len(xl_df)): 
#         prot1, res1, prot2, res2 = xl_df.iloc[i,0:4]

#         sel1 = IMP.atom.Selection(h, molecule=prot1, residue_index=res1)
#         pid1 = sel1.get_selected_particle_indexes()[0]
#         xyz1 = IMP.core.XYZ(m, pid1).get_coordinates()

#         sel2 = IMP.atom.Selection(h, molecule=prot2, residue_index=res2)
#         pid2 = sel2.get_selected_particle_indexes()[0]
#         xyz2 = IMP.core.XYZ(m, pid).get_coordinates()    

#         dist = np.linalg.norm(np.array(xyz1)-np.array(xyz2))
#         if dist < 35: 
#             xl_df.iloc[i, xl_df.columns.get_loc("sat")] = True
#         else: 
#             xl_df.iloc[i, xl_df.columns.get_loc("sat")] = True
    
# del fh


In [ ]:
n_contains = dict()
n_satisfied = dict() 
n_xls = dict() 

prots = ["MTOR", "RICTOR", "MSIN1", "MLST8", "AKT1"]
for prot in prots: 
    for prot2 in prots: 
        entry = prot + "|" + prot2
        if prot2 + "|" + prot not in n_contains.keys(): 
            n_contains[entry] = 0
            n_satisfied[entry] = 0
            n_xls[entry] = 0

for i in range(len(xl_df)): 
    prot1 = xl_df.iloc[i, xl_df.columns.get_loc('prot1')]
    prot2 = xl_df.iloc[i, xl_df.columns.get_loc('prot2')]
    
    if prot1 == "CRIM" or prot1 == "RBD" or prot1 == "MSIN1PH": 
        prot1 = "MSIN1"
    if prot2 == "CRIM" or prot2 == "RBD" or prot2 == "MSIN1PH": 
        prot2 = "MSIN1"
        
    if prot1 == "KINASE" or prot1 == "AKT1PH": 
        prot1 = "AKT1"
    if prot2 == "KINASE" or prot2 == "AKT1PH": 
        prot2 = "AKT1"
    
    entry = prot1 + "|" + prot2
    if entry not in n_contains.keys(): 
        entry = prot2 + "|" + prot1

    n_xls[entry] = n_xls[entry] + 1

#     if xl_df.iloc[i, xl_df.columns.get_loc('contains')]: 
#         n_contains[entry] = n_contains[entry] + 1
        
    if xl_df.iloc[i, xl_df.columns.get_loc('sat')]:
        n_satisfied[entry] = n_satisfied[entry] + 1 
        
fig, ax = plt.subplots(figsize=(25, 18), dpi=80) 
labels = list(n_contains.keys())
# contains_list = [n_contains[key] for key in n_contains.keys()]
satisfied_list = [n_satisfied[key] for key in n_contains.keys()]
n_xls_list = [n_xls[key] for key in n_contains.keys()]

x = np.arange(len(labels))  # the label locations
width = .8  # the width of the bars

rects3 = ax.bar(x, n_xls_list, width/2, label='Total')
# rects1 = ax.bar(x + width/3, contains_list, width/3, label='Contains')
rects2 = ax.bar(x + width/2, satisfied_list, width/2, label='Satisfied')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Counts')
ax.set_title('XL Counts')
ax.set_xticks(x, labels)
ax.set_xticklabels(labels, rotation=45)
ax.legend()

# ax.bar_label(rects1, padding=1)
ax.bar_label(rects2, padding=1)
ax.bar_label(rects3, padding=1)

fig.tight_layout()
plt.show()


In [ ]:
len(samp_clust_B_df)

In [ ]:
print(clust_precision / (len(samp_clust_A_df) + len(samp_clust_B_df) - 2))

In [ ]:
rmf_file_tmp = Path(Path.home(), "mtorc2/exp_8/66/analysis/analysis.367036/clust0.rmf3")
fh_tmp = RMF.open_rmf_file_read_only(str(rmf_file_tmp))
m_tmp = IMP.Model()
h_tmp = IMP.rmf.create_hierarchies(fh_tmp, m_tmp)[0]


In [ ]:
xl_dist_file = Path("~/mtorc2/exp_8/66/analysis/analysis.367036/traj/XLs_dist_info_0.csv")
pd.read_csv(xl_dist_file)


In [ ]:
### Write a rmf 

#     states = IMP.atom.get_by_type(h,IMP.atom.STATE_TYPE)
#     s = states[0]
    
#     output_rmf_file = Path(Path.home(), "mtorc2/exp_8/66/analysis/analysis.367036/clust_0/{}.rmf3".format(hdbscan_id))
#     fh_out = RMF.create_rmf_file(str(output_rmf_file))

#     p = IMP.Particle(m,'System')
#     hier_temp = IMP.atom.Hierarchy.setup_particle(p)
#     hier_temp.add_child(s)
#     IMP.rmf.add_hierarchy(fh_out, hier_temp)

    
#     print(hdbscan_id, rmf_id, rmsd)
#     IMP.rmf.save_frame(fh_out, str(i))

#     del fh
#     del fh_out



### Check to see if there are any anomolous RMSDS.

In [1]:
import pandas as pd
from pathlib import Path
import IMP 
import IMP.atom
import IMP.rmf
import RMF
import random
import numpy as np
import matplotlib.pyplot as plt


In [2]:
rmf_file_clust = Path(Path.home(), "mtorc2/exp_12/113/analysis/374759/sampcon_0/1/cluster.0/cluster_center_model.rmf3")
fh_clust = RMF.open_rmf_file_read_only(str(rmf_file_clust))
m_clust = IMP.Model()
h_clust = IMP.rmf.create_hierarchies(fh_clust, m_clust)[0]
    

In [3]:
rmf_file_edit = Path(Path.home(), "mtorc2/exp_12/113/analysis/374759/sampcon_0/gsms/1000_edit/B.rmf3")
fh_out = RMF.create_rmf_file(str(rmf_file_edit))


In [4]:
rmf_file = Path(Path.home(), "mtorc2/exp_12/113/analysis/374759/sampcon_0/gsms/1000/B.rmf3")
fh = RMF.open_rmf_file_read_only(str(rmf_file))
m = IMP.Model()
h = IMP.rmf.create_hierarchies(fh, m)[0]

IMP.rmf.add_hierarchy(fh_out, h)

# for mol in ["MTOR", "RICTOR", "MLST8", "MSIN1", "CRIM", "RBD", "MSIN1PH", "AKT1PH", "KINASE"]:
for i in range(1000):
    IMP.rmf.load_frame(fh, RMF.FrameID(i))

    sel_1 = IMP.atom.Selection(h_clust, resolution=1, molecule="AKT1PH", copy_index=0)
    sel_2 = IMP.atom.Selection(h, resolution=1, molecule="AKT1PH", copy_index=0)

    pids_1 = [pid for pid in sel_1.get_selected_particle_indexes()]
    pids_2 = [pid for pid in sel_2.get_selected_particle_indexes()]

    xyzs_1 = [IMP.core.XYZ(m_clust, pid) for pid in pids_1]
    xyzs_2 = [IMP.core.XYZ(m, pid) for pid in pids_2]

#         t = IMP.core.get_transformation_aligning_first_to_second(
#             xyzs_2, xyzs_1)

    rmsd = IMP.atom.get_rmsd(xyzs_2, xyzs_1)
    if rmsd > 200: 
        print(i, rmsd)
    else: 
        print(i)
        IMP.rmf.save_frame(fh_out)

del fh
del fh_out


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164 259.0435045927234
165 253.7909853125599
166 253.086823483467
167 269.61679627333206
168 255.57136912429618
169 253.086823483467
170 251.95834223887363
171 256.567394065036
172 256.567394065036
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
2

In [ ]:
fh = RMF.open_rmf_file_read_only(str(rmf_file_edit))
m = IMP.Model()
h = IMP.rmf.create_hierarchies(fh, m)[0]

for i in range(1000):
    IMP.rmf.load_frame(fh, RMF.FrameID(i))

    sel_1 = IMP.atom.Selection(h_clust, resolution=1)
    sel_2 = IMP.atom.Selection(h, resolution=1)

    pids_1 = [pid for pid in sel_1.get_selected_particle_indexes()]
    pids_2 = [pid for pid in sel_2.get_selected_particle_indexes()]

    xyzs_1 = [IMP.core.XYZ(m_clust, pid) for pid in pids_1]
    xyzs_2 = [IMP.core.XYZ(m, pid) for pid in pids_2]

#         t = IMP.core.get_transformation_aligning_first_to_second(
#             xyzs_2, xyzs_1)

    rmsd = IMP.atom.get_rmsd(xyzs_2, xyzs_1)
    print(rmsd)
#     print(i, rmsd)

del fh


In [ ]:
rmf_file = Path(Path.home(), "mtorc2/exp_12/113/analysis/374759/sampcon_0/gsms/1000/A.rmf3")
fh = RMF.open_rmf_file_read_only(str(rmf_file))
m = IMP.Model()
h = IMP.rmf.create_hierarchies(fh, m)[0]

for mol in ["MTOR", "RICTOR", "MLST8", "MSIN1", "CRIM", "RBD", "MSIN1PH", "AKT1PH", "KINASE"]:
    avg_rmsd = 0 
    for i in range(1, 100):
        IMP.rmf.load_frame(fh, RMF.FrameID(i))

#         sel_1 = IMP.atom.Selection(h_clust, resolution=1)
#         sel_2 = IMP.atom.Selection(h, resolution=1)
        sel_1 = IMP.atom.Selection(h_clust, resolution=1, molecule=mol, copy_index=0)
        sel_2 = IMP.atom.Selection(h, resolution=1, molecule=mol, copy_index=0)

        pids_1 = [pid for pid in sel_1.get_selected_particle_indexes()]
        pids_2 = [pid for pid in sel_2.get_selected_particle_indexes()]

        xyzs_1 = [IMP.core.XYZ(m_clust, pid) for pid in pids_1]
        xyzs_2 = [IMP.core.XYZ(m, pid) for pid in pids_2]

#         t = IMP.core.get_transformation_aligning_first_to_second(
#             xyzs_2, xyzs_1)

        rmsd = IMP.atom.get_rmsd_transforming_first(t, xyzs_2, xyzs_1)
        avg_rmsd = avg_rmsd + rmsd
    #     print(i, rmsd)

    print(mol, avg_rmsd / 100)
del fh
